## Download Libraries

In [ ]:
pip install moviepy

In [ ]:
pip install SpeechRecognition pydub

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [ ]:
pip install gtts

In [ ]:
pip install pytube

## URL to Video

In [ ]:
import requests
from pytube import YouTube

def download(link):

    if 'youtu.be/' in link:
        try:
            yt = YouTube(link)
            video = yt.streams.first()
            title=video.download()
            title=title.split(sep='/')[-1]
        except:
            print('failed')
            title=None

    else:
        if 'drive.google.com/' in link:
            id=link.split(sep='/')[-2]
            link='https://drive.google.com/u/0/uc?id='+id+'&export=download'
            print('sec 1')

        r = requests.get(link, allow_redirects=True)
        content_type=r.headers.get('content-type')

        if  content_type.split(sep='/')[0]=='video':
            open('video.mp4', 'wb').write(r.content)
            title='video.mp4'
        else:
            title=None
           
    return title

## Video to Audio

In [ ]:
import os
import moviepy.editor as mp
from pydub import AudioSegment

def video_to_audio(title):
    # read  Video File
    clip = mp.VideoFileClip(title)

    # write Audio File
    clip.audio.write_audiofile(r"audio.mp3")

    sound = AudioSegment.from_mp3("audio.mp3")
    sound.export("audio.wav", format="wav")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3342336/45929032 bytes (7.3%)6356992/45929032 bytes (13.8%)9396224/45929032 bytes (20.5%)12492800/45929032 bytes (27.2%)15720448/45929032 bytes (34.2%)18907136/45929032 bytes (41.2%)22126592/45929032 bytes (48.2%)25321472/45929032 bytes (55.1%)28278784/45929032 bytes (61.6%)31416320/45929032 bytes (68.4%)34414592/45929032 bytes (74.9%)37404672/45929032 bytes (81.4%)

## Audio to Text

https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python

In [ ]:
#---------------For Large Files---------------#

import speech_recognition as sr
r = sr.Recognizer()

from pydub.silence import split_on_silence

# a function that splits the audio file into chunks and applies speech recognition on these chunks
def get_audio_transcription():
    path="audio.wav"
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                None
                #print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                #print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

## Text Summarization

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS 
from string import punctuation
punctuation += '\n'
def summarizer(text):

    stopwords=list(STOP_WORDS)

    nlp = spacy.load('en_core_web_sm')

    doc=nlp(text)

    tokens= [token.text for token in doc]

    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text]=1
            else:
                word_frequencies[word.text]+=1

    max_frequency = max(word_frequencies.values())

    #Normalizing the word frequency
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word]/max_frequency

    sentence_tokens=[sent for sent in doc.sents]

    # weighing the sentence_tokens
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()]

    from heapq import nlargest
    percentage=0.30
    select_length = int(len(sentence_tokens)*percentage)

    summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
    final_summary = [word.text for word in summary]
    summary = ' '.join(final_summary)

    return summary

## Text to Audio

In [ ]:
import gtts

def text_to_audio(text):
    # make request to google to get synthesis
    tts = gtts.gTTS(text)
    # save the audio file
    tts.save("summary.mp3")

## Driver Code

In [ ]:
#driver code

link=input("Link: ")
title=download(link)
video_to_audio(title)
text = get_audio_transcription()
summary = summarizer(text)
text_to_audio(summary)

#download 'summary.mp3'

[MoviePy] Writing audio in audio.mp3


100%|██████████| 24005/24005 [00:24<00:00, 960.90it/s]

[MoviePy] Done.
